In [1]:
import numpy as np
import pandas as pd

In [2]:
with open('./eval.txt', 'r') as f:
    results = f.readlines()

# Remove any non-result lines from the eval file, and split the lines on the tab character
# (results have format: model_name\tdataset_name\tmetric_name\tmetric_value)
results = [r.replace('\n','').split('\t') for r in results if '\t' in r]

In [3]:
def remove_underscore_after(val): # Remove underscore, and keep the part after the underscore
    return val.split('_')[-1]
def remove_underscore_before(val): # Remove underscore, and keep the part before the underscore
    return val.split('_')[0]
def map2d(func, grid): # Mapping for 2d arrays, from: https://stackoverflow.com/questions/70742445/elegant-map-over-2d-list
    return [[func(value) for value in row] for row in grid]
def full_display(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(df)

df = pd.DataFrame(results, columns = ['model', 'dataset', 'metric', 'value'])
df['value'] = pd.to_numeric(df['value'])
df['model'] = df['model'].map(lambda x: '-'.join(x.split('-')[2:]))
models = df['model'].str.split('-').tolist()

# Remove all underscores from our 2d list, keep one list of the param names and one with param vals
model_names_list = map2d(remove_underscore_before, models)
model_names = np.array(model_names_list)
model_val_list = map2d(remove_underscore_after, models)
model_vals = np.array(model_val_list)

In [4]:
model_vals.shape, model_names.shape

((10802, 13), (10802, 13))

In [5]:
# Splitting model name into more columns
for i in range(model_names.shape[-1]):
    name = model_names[0][i]
    val = model_vals[:,i]
    df[name] = val
    try:
       df[name] = pd.to_numeric(df[name]) 
    except:
        pass

# Remove columns that aren't relevant
df = df.drop(['fold', 'PL', 'vit', 'model', 'data', 'ALL', 'ratio', 'kw'], axis = 1) #'method', 'AL.iter', 

# Replace 'None' in the AL iter row with NaN, to allow conversion to numerical
df['AL.iter'] = df['AL.iter'].replace('None', np.nan)
df['AL.iter'] = pd.to_numeric(df['AL.iter'])

cols = sorted(df.columns.tolist())
print(cols)
# The ratio have the format: 'ratio_xxx' where xxx is some float.
# We only want that float value, and need to convert it from string to float.
# df['ratio'] = pd.to_numeric(df['ratio'])
display(df)

# Group by all columns except the performance values, then compute mean, std and number of model runs for the performance
df_grouped = df.groupby(list(set(cols)-set(['value'])), dropna = False).agg({'value':['mean', 'std', 'count']})

['AL.iter', 'bs', 'dataset', 'epochs', 'lr', 'method', 'metric', 'value']


,dataset,metric,value,method,AL.iter,epochs,lr,bs
0,RSICD-CLS,zeroshot-val-top1,60.99,base,NaN,5,0.00005,64
1,UCM-CLS,zeroshot-val-top1,56.84,base,NaN,5,0.00005,64
2,RSICD,image_to_text_R@1,4.76,base,NaN,5,0.00005,64
3,RSICD,image_to_text_R@5,20.05,base,NaN,5,0.00005,64
4,RSICD,image_to_text_R@10,33.88,base,NaN,5,0.00005,64
...,...,...,...,...,...,...,...,...
10797,Sydney,image_to_text_R@5,8.45,base,20.0,5,0.00005,64
10798,Sydney,image_to_text_R@10,13.88,base,20.0,5,0.00005,64
10799,Sydney,text_to_image_R@1,2.62,base,20.0,5,0.00005,64
10800,Sydney,text_to_image_R@5,9.86,base,20.0,5,0.00005,64


In [6]:
display(df_grouped)

value  \
                                                                  mean   
metric            lr       AL.iter dataset method epochs bs              
image_to_text_R@1 0.000005 NaN     RSICD   ours   5      64   3.966000   
                                                         128  3.114444   
                                                  10     64   4.764000   
                                                         128  4.163333   
                                                  15     64   4.828000   
...                                                                ...   
zeroshot-val-top1 0.000500 NaN     UCM-CLS ours   25     128  6.191111   
                                                  30     64   5.333333   
                                                         128  5.856667   
                                                  35     64   5.380000   
                                                         128  5.906667   

                                                                              
                                                                   std count  
metric            lr       AL.iter dataset method epochs bs                   
image_to_text_R@1 0.000005 NaN     RSICD   ours   5      64   0.404261    10  
                                                         128  0.525288     9  
                                                  10     64   0.684044    10  
                                                         128  0.439886     9  
                                                  15     64   0.404991    10  
...                                                                ...   ...  
zeroshot-val-top1 0.000500 NaN     UCM-CLS ours   25     128  2.040695     9  
                                                  30     64   0.714598     9  
                                                         128  2.621769     9  
                                                  35     64   1.664527     9  
                                                         128  2.690938     9  

[1220 rows x 3 columns]

In [7]:
df_grouped.to_csv('eval.csv')

In [8]:
df_results = df_grouped.reset_index()
df_results = df_results.sort_values(['metric', 'dataset', ('value', 'mean')])
# df_2 = df_2[(df_2['method'] == 'ours') & (df_2['ratio'] == 0.2) & (df_2['PL'] == 'ot.image')] # (df_2['metric'] == 'image_to_text_R@5') &
# df_2 = df_2[(df_2['epochs'] > 20)  & (df_2['bs'] == 64) & (df_2['lr']==5e-5)] # &(df_2['lr']==5e-5)
# df_2 = df_2[(df_2['lr']==5e-5) & (df_2['bs'] == 64) & (df_2['method'] == 'base')] # &(df_2['lr']==5e-5)
display(df_results)

metric       lr AL.iter  dataset method epochs   bs  \
                                                                       
141   image_to_text_R@1  0.00050     NaN    RSICD   ours      5   64   
142   image_to_text_R@1  0.00050     NaN    RSICD   ours      5  128   
149   image_to_text_R@1  0.00050     NaN    RSICD   ours     25   64   
151   image_to_text_R@1  0.00050     NaN    RSICD   ours     30   64   
143   image_to_text_R@1  0.00050     NaN    RSICD   ours     10   64   
...                 ...      ...     ...      ...    ...    ...  ...   
1185  zeroshot-val-top1  0.00005     NaN  UCM-CLS   ours     35  128   
1168  zeroshot-val-top1  0.00005     NaN  UCM-CLS   base     20   64   
1169  zeroshot-val-top1  0.00005     NaN  UCM-CLS   base     25   64   
1171  zeroshot-val-top1  0.00005     NaN  UCM-CLS   base     35   64   
1170  zeroshot-val-top1  0.00005     NaN  UCM-CLS   base     30   64   

          value                  
           mean       std count  
141    0.070000  0.060000     9  
142    0.080000  0.030000     9  
149    0.080000  0.030000     9  
151    0.090000  0.045000     9  
143    0.100000  0.030000     9  
...         ...       ...   ...  
1185  65.145556  1.852924     9  
1168  66.242222  2.822781     9  
1169  66.857778  3.878353     9  
1171  67.378889  2.900217     9  
1170  67.712222  2.909960     9  

[1220 rows x 10 columns]

In [9]:
df_results.shape

(1220, 10)

In [10]:
# Results with active learning
full_display(df_results[df_results['AL.iter']>0])

metric       lr AL.iter    dataset method epochs  bs  \
                                                                         
135    image_to_text_R@1  0.00050     3.0      RSICD   base     10  64   
138    image_to_text_R@1  0.00050     5.0      RSICD   base     10  64   
132    image_to_text_R@1  0.00050     1.0      RSICD   base     10  64   
63     image_to_text_R@1  0.00005    10.0      RSICD   base      5  64   
42     image_to_text_R@1  0.00005     1.0      RSICD   base     10  64   
66     image_to_text_R@1  0.00005    20.0      RSICD   base      5  64   
43     image_to_text_R@1  0.00005     1.0      RSICD   base     15  64   
44     image_to_text_R@1  0.00005     1.0      RSICD   base     20  64   
58     image_to_text_R@1  0.00005     5.0      RSICD   base     20  64   
57     image_to_text_R@1  0.00005     5.0      RSICD   base     15  64   
51     image_to_text_R@1  0.00005     3.0      RSICD   base     15  64   
52     image_to_text_R@1  0.00005     3.0      RSICD   base     20  64   
136    image_to_text_R@1  0.00050     3.0     Sydney   base     10  64   
139    image_to_text_R@1  0.00050     5.0     Sydney   base     10  64   
133    image_to_text_R@1  0.00050     1.0     Sydney   base     10  64   
67     image_to_text_R@1  0.00005    20.0     Sydney   base      5  64   
60     image_to_text_R@1  0.00005     5.0     Sydney   base     20  64   
53     image_to_text_R@1  0.00005     3.0     Sydney   base     15  64   
64     image_to_text_R@1  0.00005    10.0     Sydney   base      5  64   
45     image_to_text_R@1  0.00005     1.0     Sydney   base     10  64   
59     image_to_text_R@1  0.00005     5.0     Sydney   base     15  64   
54     image_to_text_R@1  0.00005     3.0     Sydney   base     20  64   
46     image_to_text_R@1  0.00005     1.0     Sydney   base     15  64   
47     image_to_text_R@1  0.00005     1.0     Sydney   base     20  64   
140    image_to_text_R@1  0.00050     5.0        UCM   base     10  64   
134    image_to_text_R@1  0.00050     1.0        UCM   base     10  64   
137    image_to_text_R@1  0.00050     3.0        UCM   base     10  64   
48     image_to_text_R@1  0.00005     1.0        UCM   base     10  64   
49     image_to_text_R@1  0.00005     1.0        UCM   base     15  64   
65     image_to_text_R@1  0.00005    10.0        UCM   base      5  64   
55     image_to_text_R@1  0.00005     3.0        UCM   base     15  64   
68     image_to_text_R@1  0.00005    20.0        UCM   base      5  64   
50     image_to_text_R@1  0.00005     1.0        UCM   base     20  64   
62     image_to_text_R@1  0.00005     5.0        UCM   base     20  64   
61     image_to_text_R@1  0.00005     5.0        UCM   base     15  64   
56     image_to_text_R@1  0.00005     3.0        UCM   base     20  64   
318   image_to_text_R@10  0.00050     3.0      RSICD   base     10  64   
321   image_to_text_R@10  0.00050     5.0      RSICD   base     10  64   
315   image_to_text_R@10  0.00050     1.0      RSICD   base     10  64   
225   image_to_text_R@10  0.00005     1.0      RSICD   base     10  64   
226   image_to_text_R@10  0.00005     1.0      RSICD   base     15  64   
227   image_to_text_R@10  0.00005     1.0      RSICD   base     20  64   
246   image_to_text_R@10  0.00005    10.0      RSICD   base      5  64   
249   image_to_text_R@10  0.00005    20.0      RSICD   base      5  64   
235   image_to_text_R@10  0.00005     3.0      RSICD   base     20  64   
241   image_to_text_R@10  0.00005     5.0      RSICD   base     20  64   
234   image_to_text_R@10  0.00005     3.0      RSICD   base     15  64   
240   image_to_text_R@10  0.00005     5.0      RSICD   base     15  64   
319   image_to_text_R@10  0.00050     3.0     Sydney   base     10  64   
322   image_to_text_R@10  0.00050     5.0     Sydney   base     10  64   
316   image_to_text_R@10  0.00050     1.0     Sydney   base     10  64   
250   image_to_text_R@10  0.00005    20.0     Sydney   base      5  64   
243   image_to_text_R@

In [11]:
# Results for base CLIP model (filtered a bit)
full_display(df_results[(df_results['AL.iter'].isna()) & (df_results['method'] == 'base') & (df_results['lr']==5e-5)])

metric       lr AL.iter    dataset method epochs  bs  \
                                                                         
69     image_to_text_R@1  0.00005     NaN      RSICD   base      5  64   
71     image_to_text_R@1  0.00005     NaN      RSICD   base     15  64   
70     image_to_text_R@1  0.00005     NaN      RSICD   base     10  64   
72     image_to_text_R@1  0.00005     NaN      RSICD   base     20  64   
73     image_to_text_R@1  0.00005     NaN      RSICD   base     25  64   
75     image_to_text_R@1  0.00005     NaN      RSICD   base     35  64   
74     image_to_text_R@1  0.00005     NaN      RSICD   base     30  64   
90     image_to_text_R@1  0.00005     NaN     Sydney   base      5  64   
91     image_to_text_R@1  0.00005     NaN     Sydney   base     10  64   
92     image_to_text_R@1  0.00005     NaN     Sydney   base     15  64   
93     image_to_text_R@1  0.00005     NaN     Sydney   base     20  64   
94     image_to_text_R@1  0.00005     NaN     Sydney   base     25  64   
96     image_to_text_R@1  0.00005     NaN     Sydney   base     35  64   
95     image_to_text_R@1  0.00005     NaN     Sydney   base     30  64   
111    image_to_text_R@1  0.00005     NaN        UCM   base      5  64   
112    image_to_text_R@1  0.00005     NaN        UCM   base     10  64   
113    image_to_text_R@1  0.00005     NaN        UCM   base     15  64   
115    image_to_text_R@1  0.00005     NaN        UCM   base     25  64   
114    image_to_text_R@1  0.00005     NaN        UCM   base     20  64   
117    image_to_text_R@1  0.00005     NaN        UCM   base     35  64   
116    image_to_text_R@1  0.00005     NaN        UCM   base     30  64   
252   image_to_text_R@10  0.00005     NaN      RSICD   base      5  64   
253   image_to_text_R@10  0.00005     NaN      RSICD   base     10  64   
254   image_to_text_R@10  0.00005     NaN      RSICD   base     15  64   
255   image_to_text_R@10  0.00005     NaN      RSICD   base     20  64   
257   image_to_text_R@10  0.00005     NaN      RSICD   base     30  64   
256   image_to_text_R@10  0.00005     NaN      RSICD   base     25  64   
258   image_to_text_R@10  0.00005     NaN      RSICD   base     35  64   
273   image_to_text_R@10  0.00005     NaN     Sydney   base      5  64   
274   image_to_text_R@10  0.00005     NaN     Sydney   base     10  64   
275   image_to_text_R@10  0.00005     NaN     Sydney   base     15  64   
276   image_to_text_R@10  0.00005     NaN     Sydney   base     20  64   
277   image_to_text_R@10  0.00005     NaN     Sydney   base     25  64   
278   image_to_text_R@10  0.00005     NaN     Sydney   base     30  64   
279   image_to_text_R@10  0.00005     NaN     Sydney   base     35  64   
294   image_to_text_R@10  0.00005     NaN        UCM   base      5  64   
295   image_to_text_R@10  0.00005     NaN        UCM   base     10  64   
296   image_to_text_R@10  0.00005     NaN        UCM   base     15  64   
297   image_to_text_R@10  0.00005     NaN        UCM   base     20  64   
299   image_to_text_R@10  0.00005     NaN        UCM   base     30  64   
298   image_to_text_R@10  0.00005     NaN        UCM   base     25  64   
300   image_to_text_R@10  0.00005     NaN        UCM   base     35  64   
435    image_to_text_R@5  0.00005     NaN      RSICD   base      5  64   
436    image_to_text_R@5  0.00005     NaN      RSICD   base     10  64   
437    image_to_text_R@5  0.00005     NaN      RSICD   base     15  64   
438    image_to_text_R@5  0.00005     NaN      RSICD   base     20  64   
439    image_to_text_R@5  0.00005     NaN      RSICD   base     25  64   
440    image_to_text_R@5  0.00005     NaN      RSICD   base     30  64   
441    image_to_text_R@5  0.00005     NaN      RSICD   base     35  64   
456    image_to_text_R@5  0.00005     NaN     Sydney   base      5  64   
457    image_to_text_R@5  0.00005     NaN     Sydney   base     10  64   
458    image_to_text_R@5  0.00005     NaN     Sydney   base     15  64   
459    image_to_text_R

In [12]:
# Results for S-CLIP model (filtered a bit)
full_display(df_results[(df_results['AL.iter'].isna()) & (df_results['method'] == 'ours') & (df_results['lr']==5e-5)])

metric       lr AL.iter    dataset method epochs   bs  \
                                                                          
76     image_to_text_R@1  0.00005     NaN      RSICD   ours      5   64   
78     image_to_text_R@1  0.00005     NaN      RSICD   ours     10   64   
77     image_to_text_R@1  0.00005     NaN      RSICD   ours      5  128   
80     image_to_text_R@1  0.00005     NaN      RSICD   ours     15   64   
79     image_to_text_R@1  0.00005     NaN      RSICD   ours     10  128   
86     image_to_text_R@1  0.00005     NaN      RSICD   ours     30   64   
81     image_to_text_R@1  0.00005     NaN      RSICD   ours     15  128   
82     image_to_text_R@1  0.00005     NaN      RSICD   ours     20   64   
84     image_to_text_R@1  0.00005     NaN      RSICD   ours     25   64   
83     image_to_text_R@1  0.00005     NaN      RSICD   ours     20  128   
88     image_to_text_R@1  0.00005     NaN      RSICD   ours     35   64   
85     image_to_text_R@1  0.00005     NaN      RSICD   ours     25  128   
87     image_to_text_R@1  0.00005     NaN      RSICD   ours     30  128   
89     image_to_text_R@1  0.00005     NaN      RSICD   ours     35  128   
97     image_to_text_R@1  0.00005     NaN     Sydney   ours      5   64   
101    image_to_text_R@1  0.00005     NaN     Sydney   ours     15   64   
103    image_to_text_R@1  0.00005     NaN     Sydney   ours     20   64   
98     image_to_text_R@1  0.00005     NaN     Sydney   ours      5  128   
99     image_to_text_R@1  0.00005     NaN     Sydney   ours     10   64   
107    image_to_text_R@1  0.00005     NaN     Sydney   ours     30   64   
105    image_to_text_R@1  0.00005     NaN     Sydney   ours     25   64   
109    image_to_text_R@1  0.00005     NaN     Sydney   ours     35   64   
100    image_to_text_R@1  0.00005     NaN     Sydney   ours     10  128   
106    image_to_text_R@1  0.00005     NaN     Sydney   ours     25  128   
104    image_to_text_R@1  0.00005     NaN     Sydney   ours     20  128   
102    image_to_text_R@1  0.00005     NaN     Sydney   ours     15  128   
108    image_to_text_R@1  0.00005     NaN     Sydney   ours     30  128   
110    image_to_text_R@1  0.00005     NaN     Sydney   ours     35  128   
118    image_to_text_R@1  0.00005     NaN        UCM   ours      5   64   
119    image_to_text_R@1  0.00005     NaN        UCM   ours      5  128   
120    image_to_text_R@1  0.00005     NaN        UCM   ours     10   64   
121    image_to_text_R@1  0.00005     NaN        UCM   ours     10  128   
122    image_to_text_R@1  0.00005     NaN        UCM   ours     15   64   
124    image_to_text_R@1  0.00005     NaN        UCM   ours     20   64   
126    image_to_text_R@1  0.00005     NaN        UCM   ours     25   64   
123    image_to_text_R@1  0.00005     NaN        UCM   ours     15  128   
128    image_to_text_R@1  0.00005     NaN        UCM   ours     30   64   
130    image_to_text_R@1  0.00005     NaN        UCM   ours     35   64   
125    image_to_text_R@1  0.00005     NaN        UCM   ours     20  128   
127    image_to_text_R@1  0.00005     NaN        UCM   ours     25  128   
129    image_to_text_R@1  0.00005     NaN        UCM   ours     30  128   
131    image_to_text_R@1  0.00005     NaN        UCM   ours     35  128   
259   image_to_text_R@10  0.00005     NaN      RSICD   ours      5   64   
261   image_to_text_R@10  0.00005     NaN      RSICD   ours     10   64   
260   image_to_text_R@10  0.00005     NaN      RSICD   ours      5  128   
263   image_to_text_R@10  0.00005     NaN      RSICD   ours     15   64   
262   image_to_text_R@10  0.00005     NaN      RSICD   ours     10  128   
265   image_to_text_R@10  0.00005     NaN      RSICD   ours     20   64   
267   image_to_text_R@10  0.00005     NaN      RSICD   ours     25   64   
264   image_to_text_R@10  0.00005     NaN      RSICD   ours     15  128   
269   image_to_text_R@10  0.00005     NaN      RSICD   ours     30   64   
271   image_to_text_R@10  0.00005     NaN  